In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
# from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import itertools
from collections import defaultdict
from shapely.geometry import LineString, Point
import copy
import graphviz
from Bio import Phylo
from io import StringIO 

In [6]:
import sys
sys.path.append('../')
import Image

# Klasa
artybuty:
- bnw_image
- linie pionowe
- linie poziome
- orientacja drzewa - podana przez użytkownika

metody:
- convert to BnW
- niezerowe pixele
- znajdowanie linii poziomych i pionowych
- znajdowanie przecieć
- znajdowanie liści
- przekrecanie drzewa (ewentualnie?)

In [7]:
import cv2
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import itertools
from collections import defaultdict
from shapely.geometry import LineString, Point

class Image:
    def __init__(
            self,
            image_path,
            orientation="horizontal",
    ):
        self.image_path = image_path
        self.orientation = orientation

        self.BnW_image = self.convert_to_bnw()
        self.nonzero_pixels = self.get_nonzero_pixels()

    def convert_to_bnw(self):
        image = cv2.imread(self.image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        (thresh, BnW_image) = cv2.threshold(gray_image, 125, 255, cv2.THRESH_BINARY)
        BnW_image = np.where(BnW_image == 255, 0, 255)
        return BnW_image

    def get_nonzero_pixels(self):
        nonzero_pixels = []
        for y, x in zip(list(self.BnW_image.nonzero()[0]), list(self.BnW_image.nonzero()[1])):
            nonzero_pixels.append((x, y))
        return nonzero_pixels

    def get_top_coordinates(
            self,
            coord="y",
            min_freq=10,
    ):
        if coord == "x":
            occurrences = Counter([x for x, y in self.nonzero_pixels])
        elif coord == "y":
            occurrences = Counter([y for x, y in self.nonzero_pixels])
        selected_points = [point for point, freq in occurrences.items() if freq >= min_freq]
        print(f"Number of {coord} points with frequency > {min_freq}: {len(selected_points)}")
        return selected_points

    def find_line_coords_old(
            self,
            candidate,
            orientation='h',
            max_gap=2,
            min_line_length=20
    ):
        x_candidates = sorted([x for x, y in self.nonzero_pixels if y == candidate])

        i = 0
        line_coords = []
        x0 = x_candidates[0]

        while i < len(x_candidates) - 1:
            if x_candidates[i + 1] - x_candidates[i] > max_gap:
                if x_candidates[i] - x0 >= min_line_length:
                    line_coords.append([(x0, candidate), (x_candidates[i], candidate)])
                x0 = x_candidates[i + 1]
                break
            i += 1

        if x_candidates[i] - x0 >= min_line_length:
            line_coords.append([(x0, candidate), (x_candidates[i], candidate)])

        return line_coords

    def find_line_coords_y(
        self,
        candidate,
        max_gap=2,
        min_line_length=20
    ):
#         print(f"y candidate = {candidate}")
        x_candidates = sorted([x for x, y in image.nonzero_pixels if y == candidate])
      
        lines = []
        line = [x_candidates[0]]
        for x0, x1 in zip(x_candidates[:-1], x_candidates[1:]):
            if x1 - x0 <= max_gap:
                line.append(x1)
            else:
                line = []
            if len(line) >= min_line_length and line not in lines:
                lines.append(line)
        lines_coords = [[(line[0], candidate), (line[-1], candidate)] for line in lines]
        return lines_coords

    def find_line_coords_x(
        self,
        candidate,
        max_gap=2,
        min_line_length=20
    ):
#         print(f"x candidate = {candidate}")
        y_candidates = sorted([y for x, y in image.nonzero_pixels if x == candidate])
      
        lines = []
        line = [y_candidates[0]]
        for y0, y1 in zip(y_candidates[:-1], y_candidates[1:]):
            if y1 - y0 <= max_gap:
                line.append(y1)
            else:
                line = []
            if len(line) >= min_line_length and line not in lines:
                lines.append(line)
        lines_coords = [[(candidate, line[0]), (candidate, line[-1])] for line in lines]
        return lines_coords
    
    def find_all_lines(
            self,
            max_gap=2,
            min_line_length=20,
            min_freq=50,
#             threshold=2,
    ):
        y_candidates = self.get_top_coordinates(coord='y', min_freq=min_freq)
        x_candidates = self.get_top_coordinates(coord='x', min_freq=min_freq)
        
        vertical_lines, horizontal_lines = {}, {}
        for y_candidate in y_candidates:
            horizontal_lines[y_candidate] = self.find_line_coords_y(y_candidate, max_gap, min_line_length)

        for x_candidate in x_candidates:
            vertical_lines[x_candidate] = self.find_line_coords_x(x_candidate, max_gap, min_line_length)
        
        return vertical_lines, horizontal_lines

    def merge_lines(
            self,
    ):
        pass

In [8]:
image.get_top_coordinates(coord='y', min_freq=20)

NameError: name 'image' is not defined

In [ ]:
input_path = "/home/joanna/Pulpit/Bioinf 2.2/ADP/projekt/pdf_tree_converter/test_data/dendogram2.png"
image = Image(input_path)
print(f"Nonzero = {image.nonzero_pixels[:5]}")

v_lines, h_lines = image.find_all_lines(max_gap=2, min_line_length=20, min_freq=10)

plt.figure(figsize=(10,6))
for line in list(v_lines.values()) + list(h_lines.values()):
    if line:
        x1, y1 = line[0][0]
        x2, y2 = line[0][1]
        plt.plot([x1, x2], [y1, y2], marker='o')
        
xmin, xmax, ymin, ymax = 0, 255, 0, 255
plt.ylim(ymax, ymin)
plt.show()

In [ ]:
px.imshow(BnW_image)

In [ ]:
# candidate = can[3]
# print(f"y candidate = {candidate}")
# x_candidates = sorted([x for x, y in image.nonzero_pixels if y == candidate])
# x_candidates
# min_length = 10
# max_gap = 2

# lines= []
# line = [x_candidates[0]]

# for x0, x1 in zip(x_candidates[:-1], x_candidates[1:]):
#     if x1 - x0 <= max_gap:
#         line.append(x1)
#     else:
#         line = []
#     if len(line) >= min_length and line not in lines:
#         lines.append(line)
# lines_coords = [[(line[0], candidate), (line[-1], candidate)] for line in lines]
# lines_coords

In [ ]:
# x_candidates

In [ ]:
lines = image.find_all_lines(max_gap=10, min_line_length=10)
print(lines)

plt.figure(figsize=(10,6))
for line in lines.values():
    if line:
        print(line[0][0])
        x1, y1 = line[0][0]
        x2, y2 = line[0][1]
        plt.plot([x1, x2], [y1, y2], marker='o')
        
xmin, xmax, ymin, ymax = 0, 255, 0, 255
plt.ylim(ymax, ymin)
plt.show()

In [ ]:
px.imshow(BnW_image)

In [ ]:
image.get_top_coordinates(min_freq=50, coord='x')

In [ ]:
image.nonzero_pixels

In [ ]:
px.imshow(image.BnW_image)

# Array based line detection

In [ ]:
input_path = "/home/joanna/Pulpit/Bioinf 2.2/ADP/projekt/pdf_tree_converter/test_data/dendogram2.png"

In [ ]:
# read the image
image = cv2.imread(input_path)

#Convert an image from BGR to grayscale mode 
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
#Convert a grayscale image to black and white using binary thresholding 
(thresh, BnW_image) = cv2.threshold(gray_image, 125, 255, cv2.THRESH_BINARY)

BnW_image = np.where(BnW_image==255, 0, 255)

#display all the images
# fig = px.imshow(image, title='Original image')
# fig.show()

# fig = px.imshow(gray_image, title="Gray image")
# fig.show()

px.imshow(BnW_image, title="Black and white image")

In [ ]:
BnW_image.nonzero()